In [2]:
import nltk
from nltk.corpus import brown, treebank, gutenberg
from nltk.tree import Tree
from collections import defaultdict
from timeit import timeit

3. Consider the sentence Kim arrived or Dana left and everyone cheered. Write down the parenthesized forms to show the relative scope of and and or. Generate tree structures corresponding to both of these interpretations. 

In [21]:
#Kim arrived (or Dana left) and everyone cheered 
#(Kim arrived or Dana left) and everyone cheered

In [41]:
grammar1 = nltk.CFG.fromstring("""
... S -> NP VP | S Conj S
... PP -> P NP
... NP -> N  
... VP -> V NP | VP PP | V
... Conj -> 'and' | 'or'
... N -> 'Kim' | 'Dana' | 'everyone'
... V -> 'arrived' | 'left' | 'cheered'
... """)

In [20]:
# rd_parser = nltk.RecursiveDescentParser(grammar1, trace=2)
sent = 'Kim arrived or Dana left and everyone cheered'.split()
parser = nltk.ChartParser(grammar1)
for tree in parser.parse(sent):
     print(tree)

(S
  (S
    (S (NP (N Kim)) (VP (V arrived)))
    (Conj or)
    (S (NP (N Dana)) (VP (V left))))
  (Conj and)
  (S (NP (N everyone)) (VP (V cheered))))
(S
  (S (NP (N Kim)) (VP (V arrived)))
  (Conj or)
  (S
    (S (NP (N Dana)) (VP (V left)))
    (Conj and)
    (S (NP (N everyone)) (VP (V cheered)))))


4. The Tree class implements a variety of other useful methods. See the Tree help documentation for more details, i.e. import the Tree class and then type help(Tree).

In [22]:
help(Tree)

Help on class Tree in module nltk.tree:

class Tree(builtins.list)
 |  Tree(node, children=None)
 |  
 |  A Tree represents a hierarchical grouping of leaves and subtrees.
 |  For example, each constituent in a syntax tree is represented by a single Tree.
 |  
 |  A tree's children are encoded as a list of leaves and subtrees,
 |  where a leaf is a basic (non-tree) value; and a subtree is a
 |  nested Tree.
 |  
 |      >>> from nltk.tree import Tree
 |      >>> print(Tree(1, [2, Tree(3, [4]), 5]))
 |      (1 2 (3 4) 5)
 |      >>> vp = Tree('VP', [Tree('V', ['saw']),
 |      ...                  Tree('NP', ['him'])])
 |      >>> s = Tree('S', [Tree('NP', ['I']), vp])
 |      >>> print(s)
 |      (S (NP I) (VP (V saw) (NP him)))
 |      >>> print(s[1])
 |      (VP (V saw) (NP him))
 |      >>> print(s[1,1])
 |      (NP him)
 |      >>> t = Tree.fromstring("(S (NP I) (VP (V saw) (NP him)))")
 |      >>> s == t
 |      True
 |      >>> t[1][1].set_label('X')
 |      >>> t[1][1].label()
 

5. In this exercise you will manually construct some parse trees.

Write code to produce two trees, one for each reading of the phrase old men and women
Encode any of the trees presented in this chapter as a labeled bracketing and use nltk.Tree() to check that it is well-formed. Now use draw() to display the tree.
As in (a) above, draw a tree for The woman saw a man last Thursday.

In [7]:
phrase  = 'old men and women'.split()
grammar2 = nltk.CFG.fromstring("""
  S -> NP VP | S Conj S | N
  NP -> Det Nom | PropN
  Nom -> Adj Nom | N
  N -> "men" | "women" 
  ADJ -> "old"
  CONJ -> "and"
  """)

In [ ]:
parser = nltk.ChartParser(grammar2)
for tree in rd_parser.parse(phrase):
    print(tree)

In [42]:
phrase1_1 = Tree('NP', 
             [Tree('JJ', ['old']), 
              Tree('NP', 
                   [Tree('N', ['men']), Tree('Conj', ['and']), Tree('N', ['women'])])])
print(phrase1_1)

(NP (JJ old) (NP (N men) (Conj and) (N women)))


In [24]:
phrase1_2 = nltk.Tree('JJ', ['old']), ('NP', ['old', 'men', 'and', 'women'])
print(phrase1_2)

(Tree('JJ', ['old']), ('NP', ['old', 'men', 'and', 'women']))


In [ ]:
phrase = Tree.fromstring("(S (NP (ADJ old) (N men)) (AND and) (N (N women)))")
phrase.draw()

In [43]:
phrase3 = 'The woman saw a man last Thursday'.split()
grammar4 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> Adv NP
  V -> "saw"
  NP -> "woman" | "man" | "Thursday" | Det N 
  Det -> "a" | "The"
  N -> "man" | "woman" | "Thursday"
  Adv -> "last" 
  """)

In [37]:
rd_parser = nltk.RecursiveDescentParser(grammar4)
for tree in rd_parser.parse(phrase3):
      print(tree)

(S
  (NP (Det The) (N woman))
  (VP (V saw) (NP (Det a) (N man)) (PP (Adv last) (NP Thursday))))


In [43]:
phrase3_tree = nltk.Tree('S', (('NP', ['The', 'woman']), ('VP', ['saw'], ('NP', ['a', 'man'], ('PP', ['last'], ('NP', ['Thursday']))))          

SyntaxError: unexpected EOF while parsing (<ipython-input-43-26505f54b2c4>, line 1)

In [44]:
tree_3 = Tree.fromstring("(S (NP (Det The)(N woman)) (VP (V saw) (NP (Det a)(N man)) (PP (ADJ last)(NOM Thursday))))")
tree_3.draw()

6.Write a recursive function to traverse a tree and return the depth of the tree, such
that a tree with a single node would have depth zero. (Hint: the depth of a subtree
is the maximum depth of its children, plus one.)

In [51]:
def depth(tree):
    max_child_height = 0
    for child in tree:
        if isinstance(child, Tree):
            max_child_height = max(max_child_height, child.height())
        else:
            max_child_height = max(max_child_height, 1)
    return max_child_height
 
 
print(depth(tree))

tree = Tree('S', [Tree('ADJN', [Tree('ADJ', ['old']), Tree('N', ['men'])]), Tree('AND', ['and']),
                  Tree('N', [Tree('N', ['women'])])])

3


12.  We have seen that a chart parser adds but never removes edges from a chart. Why?

A chart is a set of edges and a location. The edge set represents the state of a chart parser during processing, and new edges can be inserted into the set at any time. Once entered, an edge cannot be modified or removed. The chart also stores a location. This represents the combined span of the list of input tokens. The chart uses this information in order to return edges which span the entire sentence. https://lost-contact.mit.edu/afs/cs.pitt.edu/projects/nltk/docs/tutorial/parsing/x385.html

13. Consider the sequence of words: Buffalo buffalo Buffalo buffalo buffalo buffalo
Buffalo buffalo. This is a grammatically correct sentence, as explained at http://en
.wikipedia.org/wiki/Buffalo_buffalo_Buffalo_buffalo_buffalo_buffalo_Buffalo_buf
falo. Consider the tree diagram presented on this Wikipedia page, and write down
a suitable grammar. Normalize case to lowercase, to simulate the problem that a
listener has when hearing this sentence. Can you find other parses for this sentence?
How does the number of parse trees grow as the sentence gets longer? (More examples
of these sentences can be found at http://en.wikipedia.org/wiki/List_of_ho
mophonous_phrases.)

In [55]:
phrase5 = 'Buffalo buffalo Buffalo buffalo buffalo buffalo Buffalo buffalo'.split()
grammar5 = nltk.CFG.fromstring("""
  S -> NP VP
  NP -> RC NP | PN N
  RC -> NP V
  VP -> V NP 
  V -> "buffalo"
  N -> "buffalo" | "Buffalo"
  PN -> "Buffalo"
  """)



15. Extend the grammar in grammar2 with productions that expand prepositions as
intransitive, transitive, and requiring a PP complement. Based on these productions,
use the method of the preceding exercise to draw a tree for the sentence Lee
ran away home.

In [3]:
grammar2 = nltk.CFG.fromstring("""
  S  -> NP VP
  NP -> Det Nom | PropN
  Nom -> Adj Nom | N
  VP -> V Adj | TV NP | TV S | VPP NP PP
  PP -> P NP
 
  PropN -> 'Buster' | 'Chatterer' | 'Joe'
  Det -> 'the' | 'a'
  N -> 'bear' | 'squirrel' | 'tree' | 'fish' | 'log'
  Adj  -> 'angry' | 'frightened' |  'little' | 'tall'
  V ->   'was' |
  TV -> 'chased'  | 'saw' | 'said' | 'thought' | 
  VPP ->  'put'
  P -> 'on'
  """)

In [12]:
sent6 = 'Lee ran away home'.split()
grammar6 = nltk.CFG.fromstring("""
  S  -> NP VP
  NP -> Det Nom | PropN
  Nom -> Adj Nom | N
  VP -> V Adj | V NP | V S | VPP PP
  PP -> P NP
 
  PropN -> 'Buster' | 'Lee'
  Det -> 'the' | 'a'
  N -> 'home' 
  Adj  -> 'angry' 
  V ->   'was' |
  TV -> 'chased'  | 'saw' | 'said' | 'thought' | 
  VPP ->  'ran'
  P -> 'away'
  """)
parser = nltk.ChartParser(grammar6)
for tree in parser.parse(sent6):
     print(tree)

16. Pick some common verbs and complete the following tasks:

Write a program to find those verbs in the Prepositional Phrase Attachment Corpus nltk.corpus.ppattach. Find any cases where the same verb exhibits two different attachments, but where the first noun, or second noun, or preposition, stay unchanged (as we saw in our discussion of syntactic ambiguity in 2).
Devise CFG grammar productions to cover some of these cases.

In [ ]:
class PPAttachment(object):
    def __init__(self, sent, verb, noun1, prep, noun2, attachment):
        self.sent = sent
        self.verb = verb
        self.noun1 = noun1
        self.prep = prep
        self.noun2 = noun2
        self.attachment = attachment

    def __repr__(self):
        return (
            "PPAttachment(sent=%r, verb=%r, noun1=%r, prep=%r, "
            "noun2=%r, attachment=%r)"
            % (self.sent, self.verb, self.noun1, self.prep, self.noun2, self.attachment)
        )

In [39]:
entries = nltk.corpus.ppattach.attachments('training')
table = nltk.defaultdict(lambda: nltk.defaultdict(set)) # empty dict
for entry in entries: #filling the dict with the keys that will contain the noun1-preposition-noun1, #values for keys are dict with keys attachment and values verb
    pphrase = entry.noun1 + '-' + entry.prep + '-' + entry.noun2
    key1 = entry.verb + '-' + entry.noun1
    table[key1][entry.attachment].add(pphrase)
    
att_list=[('see', verb_noun,table[verb_noun]) for verb_noun in table.keys() 
          if str(verb_noun).startswith('see') and len( table[verb_noun])>1]
print(att_list)

[('see', 'seeking-million', defaultdict(<class 'set'>, {'N': {'million-in-increases'}, 'V': {'million-from-unit', 'million-for-firm'}})), ('see', 'seek-majority', defaultdict(<class 'set'>, {'N': {'majority-of-seats'}, 'V': {'majority-on-board'}})), ('see', 'seeking-offers', defaultdict(<class 'set'>, {'N': {'offers-for-stations'}, 'V': {'offers-from-bidders'}})), ('see', 'see-step', defaultdict(<class 'set'>, {'N': {'step-to-trading'}, 'V': {'step-as-knell'}})), ('see', 'seek-control', defaultdict(<class 'set'>, {'N': {'control-of-Rally', 'control-of-company', 'control-of-airline', 'control-of-companies'}, 'V': {'control-of-Inc.'}})), ('see', 'seeking-buyer', defaultdict(<class 'set'>, {'V': {'buyer-for-facility', 'buyer-for-chain', 'buyer-before-1993'}, 'N': {'buyer-for-shares'}})), ('see', 'seeking-injunction', defaultdict(<class 'set'>, {'N': {'injunction-against-offer'}, 'V': {'injunction-against-bid'}})), ('see', 'seek-protection', defaultdict(<class 'set'>, {'V': {'protection-un

19. Read up on "garden path" sentences. How might the computational work of a parser relate to the difficulty humans have with processing these sentences? http://en.wikipedia.org/wiki/Garden_path_sentence

20. To compare multiple trees in a single window, we can use the draw_trees() method. Define some trees and try it out:

 	
>>> from nltk.draw.tree import draw_trees
>>> draw_trees(tree1, tree2, tree3)  

In [47]:
sent11 = 'Kim arrived or Dana left and everyone cheered'.split()
parser = nltk.ChartParser(grammar1)
for tree1 in parser.parse(sent11):
     return tree1

rd_parser = nltk.RecursiveDescentParser(grammar4)
for tree2 in rd_parser.parse(phrase3):
      return tree2

from nltk.draw.tree import draw_trees
draw_trees(tree1, tree2)

SyntaxError: 'return' outside function (<ipython-input-47-cdadcbdc2369>, line 4)

21. Using tree positions, list the subjects of the first 100 sentences in the Penn treebank; to make the results easier to view, limit the extracted subjects to subtrees whose height is 2.

In [9]:
t = treebank.parsed_sents('wsj_0001.mrg')[:100]
def filter_s(tree):
    child_nodes = [child.label() for child in tree
                   if isinstance(child, nltk.Tree) and child.height() < 2]
    return  (tree.label() == 'NP-SBJ')
[subtree for tree in treebank.parsed_sents() for subtree in tree.subtrees(filter_s)]

[Tree('NP-SBJ', [Tree('NP', [Tree('NNP', ['Pierre']), Tree('NNP', ['Vinken'])]), Tree(',', [',']), Tree('ADJP', [Tree('NP', [Tree('CD', ['61']), Tree('NNS', ['years'])]), Tree('JJ', ['old'])]), Tree(',', [','])]),
 Tree('NP-SBJ', [Tree('NNP', ['Mr.']), Tree('NNP', ['Vinken'])]),
 Tree('NP-SBJ', [Tree('-NONE-', ['*-1'])]),
 Tree('NP-SBJ', [Tree('NP', [Tree('NP', [Tree('DT', ['A']), Tree('NN', ['form'])]), Tree('PP', [Tree('IN', ['of']), Tree('NP', [Tree('NN', ['asbestos'])])])]), Tree('RRC', [Tree('ADVP-TMP', [Tree('RB', ['once'])]), Tree('VP', [Tree('VBN', ['used']), Tree('NP', [Tree('-NONE-', ['*'])]), Tree('S-CLR', [Tree('NP-SBJ', [Tree('-NONE-', ['*'])]), Tree('VP', [Tree('TO', ['to']), Tree('VP', [Tree('VB', ['make']), Tree('NP', [Tree('NNP', ['Kent']), Tree('NN', ['cigarette']), Tree('NNS', ['filters'])])])])])])])]),
 Tree('NP-SBJ', [Tree('-NONE-', ['*'])]),
 Tree('NP-SBJ', [Tree('NNS', ['researchers'])]),
 Tree('NP-SBJ', [Tree('NP', [Tree('DT', ['The']), Tree('NN', ['asbestos'])